## 2. Byte-Pair Encoding (BPE) Tokenizer

Maybe useful: https://zhuanlan.zhihu.com/p/1927397109025473129

### Problem (unicode1): Understanding Unicode (1 point)

In [1]:
ord('牛')

29275

In [2]:
chr(29275)

'牛'

(a) What Unicode character does chr(0) return?

**Deliverable**: A one-sentence response.

<span style="background-color: #29B6F6; color: black">null character</span>

In [3]:
chr(0)

'\x00'

(b) How does this character’s string representation (`__repr__()`) differ from its printed representation?

**Deliverable**: A one-sentence response.

<span style="background-color: #29B6F6; color: black">
The string representation (`__repr__()`) of the null character explicitly shows its escape sequence `'\x00'`, whereas its printed representation typically appears as an empty string or nothing at all because it is a non-printable character.
</span>

(c) What happens when this character occurs in text? It may be helpful to play around with the following in your Python interpreter and see if it matches your expectations: 

```python
>>> chr(0)
>>> print(chr(0))
>>> "this is a test" + chr(0) + "string"
>>> print("this is a test" + chr(0) + "string")
```

**Deliverable**: A one-sentence response

In [4]:
chr(0)
print(chr(0))
"this is a test" + chr(0) + "string"
print("this is a test" + chr(0) + "string")

 
this is a test string


<span style="background-color: #29B6F6; color: black">
it is embedded within the string but is typically not visible when printed.
</span>

### Problem (unicode2): Unicode Encodings (3 points)

(a) What are some reasons to prefer training our tokenizer on UTF-8 encoded bytes, rather than
UTF-16 or UTF-32? It may be helpful to compare the output of these encodings for various
input strings.

**Deliverable**: A one-to-two sentence response.

In [7]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)
print(type(utf8_encoded))
print("UTF-8 value: ", ", ".join(map(str, list(utf8_encoded))))
utf16_encoded = test_string.encode("utf-16")
print(utf16_encoded)
print(type(utf16_encoded))
print("UTF-16 value: ", ", ".join(map(str, list(utf16_encoded))))
utf32_encoded = test_string.encode("utf-32")
print(utf32_encoded)
print(type(utf32_encoded))
print("UTF-32 value: ", ", ".join(map(str, list(utf32_encoded))))

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'
<class 'bytes'>
UTF-8 value:  104, 101, 108, 108, 111, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 33
b'\xff\xfeh\x00e\x00l\x00l\x00o\x00!\x00 \x00S0\x930k0a0o0!\x00'
<class 'bytes'>
UTF-16 value:  255, 254, 104, 0, 101, 0, 108, 0, 108, 0, 111, 0, 33, 0, 32, 0, 83, 48, 147, 48, 107, 48, 97, 48, 111, 48, 33, 0
b'\xff\xfe\x00\x00h\x00\x00\x00e\x00\x00\x00l\x00\x00\x00l\x00\x00\x00o\x00\x00\x00!\x00\x00\x00 \x00\x00\x00S0\x00\x00\x930\x00\x00k0\x00\x00a0\x00\x00o0\x00\x00!\x00\x00\x00'
<class 'bytes'>
UTF-32 value:  255, 254, 0, 0, 104, 0, 0, 0, 101, 0, 0, 0, 108, 0, 0, 0, 108, 0, 0, 0, 111, 0, 0, 0, 33, 0, 0, 0, 32, 0, 0, 0, 83, 48, 0, 0, 147, 48, 0, 0, 107, 48, 0, 0, 97, 48, 0, 0, 111, 48, 0, 0, 33, 0, 0, 0


<span style="background-color: #29B6F6; color: black">
UTF-8 is the most popular way and often more space-efficient
</span>

(b) Consider the following (incorrect) function, which is intended to decode a UTF-8 byte string into
a Unicode string. Why is this function incorrect? Provide an example of an input byte string
that yields incorrect results.
``` python
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])
>>> decode_utf8_bytes_to_str_wrong("hello".encode("utf-8"))
'hello'
```

**Deliverable**: An example input byte string for which decode_utf8_bytes_to_str_wrong pro-
duces incorrect output, with a one-sentence explanation of why the function is incorrect.

In [12]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])
test_string = "hello! こんにちは!"
decode_utf8_bytes_to_str_wrong(test_string.encode("utf-8"))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 0: unexpected end of data

<span style="background-color: #29B6F6; color: black">
The function incorrectly attempts to decode each byte individually, leading to a UnicodeDecodeError because multi-byte UTF-8 characters cannot be decoded one byte at a time.
</span>

(c) Give a two byte sequence that does not decode to any Unicode character(s).

**Deliverable**: An example, with a one-sentence explanation.

In [18]:
invalid_bytes = b'\xc2\x00'
print(f"Attempting to decode: {invalid_bytes!r}")
decoded_string = invalid_bytes.decode("utf-8")
decoded_string

Attempting to decode: b'\xc2\x00'


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 0: invalid continuation byte

<span style="background-color: #29B6F6; color: black">
b'\xc2\x00'
because while 0xc2 is a valid start byte for a two-byte UTF-8 sequence, 0x00 is not a valid continuation byte.
</span>

In [22]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

# requires `regex` package
import regex as re
re.findall(PAT, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

### Problem (train_bpe): BPE Tokenizer Training (15 points)

**Deliverable**: Write a function that, given a path to an input text file, trains a (byte-level) BPE
tokenizer. Your BPE training function should handle (at least) the following input parameters:

`input_path`: str Path to a text file with BPE tokenizer training data.

`vocab_size`: int A positive integer that defines the maximum final vocabulary size (including the
initial byte vocabulary, vocabulary items produced from merging, and any special tokens).

`special_tokens`: list[str] A list of strings to add to the vocabulary. These special tokens do not
otherwise affect BPE training.

Your BPE training function should return the resulting vocabulary and merges:

`vocab`: dict[int, bytes] The tokenizer vocabulary, a mapping from int (token ID in the vocabu-
lary) to bytes (token bytes).

`merges`: list[tuple[bytes, bytes]] A list of BPE merges produced from training. Each list item
is a tuple of bytes (<token1>, <token2>), representing that <token1> was merged with
<token2>. The merges should be ordered by order of creation.

To test your BPE training function against our provided tests, you will first need to implement the
test adapter at `[adapters.run_train_bpe]`. Then, run `uv run pytest tests/test_train_bpe.py`.
Your implementation should be able to pass all tests. Optionally (this could be a large time-investment),
you can implement the key parts of your training method using some systems language, for instance
`C++` (consider `cppyy` for this) or `Rust` (using `PyO3`). If you do this, be aware of which operations
require copying vs reading directly from Python memory, and make sure to leave build instructions, or
make sure it builds using only `pyproject.toml`. Also note that the GPT-2 regex is not well-supported
in most regex engines and will be too slow in most that do. We have verified that Oniguruma is
reasonably fast and supports negative lookahead, but the `regex` package in Python is, if anything,
even faster.

In [1]:
uv run pytest tests/test_train_bpe.py

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 3 items

tests/test_train_bpe.py::test_train_bpe_speed PASSED
tests/test_train_bpe.py::test_train_bpe PASSED
tests/test_train_bpe.py::test_train_bpe_special_tokens PASSED

============================== 3 passed in 2.95s ==============================
Note: you may need to restart the kernel to use updated packages.


### Problem (train_bpe_tinystories): BPE Training on TinyStories (2 points)

Train a byte-level BPE tokenizer on the TinyStories dataset, using a maximum vocabulary size of 10,000. Make sure to add the TinyStories `<|endoftext|>` special token to the vocabulary.
Serialize the resulting vocabulary and merges to disk for further inspection. How many hours and memory did training take? What is the longest token in the vocabulary? Does it make sense?

Resource requirements: ≤30 minutes (no GPUs), ≤ 30GB RAM

**Hint** You should be able to get under 2 minutes for BPE training using multiprocessing during
pretokenization and the following two facts:

(a) The `<|endoftext|>` token delimits documents in the data files.

(b) The `<|endoftext|>` token is handled as a special case before the BPE merges are applied.

**Deliverable:** A one-to-two sentence response.

<span style="background-color: #29B6F6; color: black">
The tokenizer was trained in approximately 3 minutes (189.14 seconds) with a peak memory usage of 136.51 MB, both well within the resource limits. The longest token, a 512-byte sequence of the repeating character 's'
</span>

Profile your code. What part of the tokenizer training process takes the most time?

**Deliverable**: A one-to-two sentence response.

<span style="background-color: #29B6F6; color: black">
the pre-tokenization step.
</span>

### Problem (train_bpe_expts_owt): BPE Training on OpenWebText (2 points)

Train a byte-level BPE tokenizer on the OpenWebText dataset, using a maximum vocabulary
size of 32,000. Serialize the resulting vocabulary and merges to disk for further inspection. What
is the longest token in the vocabulary? Does it make sense?

**Resource requirements**: ≤12 hours (no GPUs), ≤ 100GB RAM

**Deliverable**: A one-to-two sentence response.

<span style="background-color: yellow; color: black">
Because it takes too long for my computer to run this, I directly used the tokenizer library of Hugging Face to generate the file
</span>

<span style="background-color: #29B6F6; color: black">
The longest token is '--------------------------------'
</span>

Compare and contrast the tokenizer that you get training on TinyStories versus OpenWebText.

**Deliverable**: A one-to-two sentence response.

<span style="background-color: #29B6F6; color: black">
TinyStories' tokenizer mainly learns simple narrative vocabulary, while OpenWebText's tokenizer contains more structural and formatted characters derived from network text.
</span>

### Problem (tokenizer): Implementing the tokenizer (15 points)

**Deliverable**: Implement a Tokenizer class that, given a vocabulary and a list of merges, encodes
text into integer IDs and decodes integer IDs into text. Your tokenizer should also support user-provided
special tokens (appending them to the vocabulary if they aren’t already there). We recommend the
following interface:

`def __init__(self, vocab, merges, special_tokens=None)` Construct a tokenizer from a given vocabulary, list of merges, and (optionally) a list of special tokens. This function should accept the following parameters:

``` python
vocab: dict[int, bytes]
merges: list[tuple[bytes, bytes]]
special_tokens: list[str] | None = None
```

`def from_files(cls, vocab_filepath, merges_filepath, special_tokens=None)` Class method that constructs and return a Tokenizer from a serialized vocabulary and list of merges (in the same format that your BPE training code output) and (optionally) a list of special tokens. This method should accept the following additional parameters:

``` python
vocab_filepath: str
merges_filepath: str
special_tokens: list[str] | None = None
```

`def encode(self, text: str) -> list[int]` Encode an input text into a sequence of token IDs.

`def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]` Given an iterable of strings (e.g., a Python file handle), return a generator that lazily yields token IDs. This is required for memory-eﬀicient tokenization of large files that we cannot directly load into memory.

`def decode(self, ids: list[int]) -> str` Decode a sequence of token IDs into text.

To test your Tokenizer against our provided tests, you will first need to implement the test adapter at `[adapters.get_tokenizer]`. Then, run `uv run pytest tests/test_tokenizer.py`. Your implementation should be able to pass all tests.

In [2]:
uv run pytest tests/test_tokenizer.py

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 25 items

tests/test_tokenizer.py::test_roundtrip_empty PASSED
tests/test_tokenizer.py::test_empty_matches_tiktoken PASSED
tests/test_tokenizer.py::test_roundtrip_single_character PASSED
tests/test_tokenizer.py::test_single_character_matches_tiktoken PASSED
tests/test_tokenizer.py::test_roundtrip_single_unicode_character PASSED
tests/test_tokenizer.py::test_single_unicode_character_matches_tiktoken PASSED
tests/test_tokenizer.py::test_roundtrip_ascii_string PASSED
tests/test_tokenizer.py::test_ascii_string_matches_tiktoken PASSED
tests/test_tokenizer.py::test_roundtrip_unicode_string PASSED
tests/test_tokenizer.py::test_unicode_string_matches_tiktoken PASSED
tests/test_tokenizer.py::test_roundtrip_unicode_string_with_special_tokens PASSED
test

### Problem (tokenizer_experiments): Experiments with tokenizers (4 points)

Sample 10 documents from TinyStories and OpenWebText. Using your previously-trained TinyStories and OpenWebText tokenizers (10K and 32K vocabulary size, respectively), encode these sampled documents into integer IDs. What is each tokenizer’s compression ratio (bytes/token)?

**Deliverable**: A one-to-two sentence response.

What happens if you tokenize your OpenWebText sample with the TinyStories tokenizer? Compare the compression ratio and/or qualitatively describe what happens.

**Deliverable**: A one-to-two sentence response.

Estimate the throughput of your tokenizer (e.g., in bytes/second). How long would it take to tokenize the Pile dataset (825GB of text)?

**Deliverable**: A one-to-two sentence response.

Using your TinyStories and OpenWebText tokenizers, encode the respective training and development datasets into a sequence of integer token IDs. We’ll use this later to train our language \text{model}. We recommend serializing the token IDs as a NumPy array of datatype uint16. Why is uint16 an appropriate choice?

**Deliverable**: A one-to-two sentence response.

---

## 3. Transformer Language Model Architecture

Maybe useful: https://zhuanlan.zhihu.com/p/1927015802915263462

<div style="display: flex; justify-content: space-around; align-items: center;">
  <img src="data/transformer.png" alt="Transformer Image" style="width: 30%;">
  <img src="data/prenorm_transformer.png" alt="Pre-Norm Transformer Image" style="width: 65%;">
</div>

### Problem (linear): Implementing the linear module (1 point)

**Deliverable**: Implement a `Linear` class that inherits from `torch.nn.Module` and performs a linear transformation. Your implementation should follow the interface of PyTorch’s built-in `nn.Linear` module, except for not having a bias argument or parameter. We recommend the following interface:

`def __init__(self, in_features, out_features, device=None, dtype=None)` Construct a
linear transformation module. This function should accept the following parameters:
``` python
in_features: int final dimension of the input
out_features: int final dimension of the output
device: torch.device | None = None Device to store the parameters on
dtype: torch.dtype | None = None Data type of the parameters
```

`def forward(self, x: torch.Tensor) -> torch.Tensor` Apply the linear transformation to the input.

Make sure to:

- subclass `nn.Module`
- call the superclass constructor
- construct and store your parameter as $W$ (not $W^T$) for memory ordering reasons, putting it in an `nn.Parameter`
- of course, don’t use `nn.Linear` or `nn.functional.linear`

For initializations, use the settings from above along with `torch.nn.init.trunc_normal_` to
initialize the weights.
To test your Linear module, implement the test adapter at `[adapters.run_linear]`. The adapter
should load the given weights into your Linear module. You can use `Module.load_state_dict` for
this purpose. Then, run `uv run pytest -k test_linear`.

In [1]:
uv run pytest -k test_linear

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_model.py::test_linear PASSED

====================== 1 passed, 47 deselected in 1.55s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (embedding): Implement the embedding module (1 point)

**Deliverable**: Implement the `Embedding` class that inherits from `torch.nn.Module` and performs an
embedding lookup. Your implementation should follow the interface of PyTorch’s built-in
`nn.Embedding` module. We recommend the following interface:

`def __init__(self, num_embeddings, embedding_dim, device=None, dtype=None)` Construct an embedding module. This function should accept the following parameters:

``` python
num_embeddings: int Size of the vocabulary
embedding_dim: int Dimension of the embedding vectors, i.e., d_model
device: torch.device | None = None Device to store the parameters on
dtype: torch.dtype | None = None Data type of the parameters
```

`def forward(self, token_ids: torch.Tensor) -> torch.Tensor` Lookup the embedding vectors for the given token IDs.

Make sure to:
- subclass nn.Module
- call the superclass constructor
- initialize your embedding matrix as a nn.Parameter
- store the embedding matrix with the d_model being the final dimension
- of course, don’t use nn.Embedding or nn.functional.embedding

Again, use the settings from above for initialization, and use `torch.nn.init.trunc_normal_` to
initialize the weights.

To test your implementation, implement the test adapter at `[adapters.run_embedding]`. Then, run
`uv run pytest -k test_embedding`.

In [2]:
uv run pytest -k test_embedding

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_model.py::test_embedding PASSED

====================== 1 passed, 47 deselected in 0.13s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (rmsnorm): Root Mean Square Layer Normalization (1 point)

**Deliverable**: Implement `RMSNorm` as a `torch.nn.Module`. We recommend the following interface:

`def __init__(self, d_model: int, eps: float = 1e-5, device=None, dtype=None)` Construct the RMSNorm module. This function should accept the following parameters:

``` python
d_model: int Hidden dimension of the model
eps: float = 1e-5 Epsilon value for numerical stability
device: torch.device | None = None Device to store the parameters on
dtype: torch.dtype | None = None Data type of the parameters
```

`def forward(self, x: torch.Tensor) -> torch.Tensor` Process an input tensor of shape (`batch_size`, `sequence_length`, `d_model`) and return a tensor of the same shape.

**Note**: Remember to upcast your input to `torch.float32` before performing the normalization (and
later downcast to the original `dtype`), as described above.

To test your implementation, implement the test adapter at `[adapters.run_rmsnorm]`. Then, run `uv
run pytest -k test_rmsnorm`.

In [3]:
uv run pytest -k test_rmsnorm

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_model.py::test_rmsnorm PASSED

====================== 1 passed, 47 deselected in 0.12s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (positionwise_feedforward): Implement the position-wise feed-forward network (2 points)

**Deliverable**: Implement the `SwiGLU` feed-forward network, composed of a `SiLU` activation function and a `GLU`.

**Note**: in this particular case, you should feel free to use torch.sigmoid in your implementation for numerical stability.

You should set dff to approximately $\frac{8}{3}$ × $d_\text{model}$ in your implementation, while ensuring that the dimensionality of the inner feed-forward layer is a multiple of 64 to make good use of your hardware. To test your implementation against our provided tests, you will need to implement
the test adapter at `[adapters.run_swiglu]`. Then, run `uv run pytest -k test_swiglu` to test your implementation.

In [4]:
uv run pytest -k test_swiglu

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_model.py::test_swiglu PASSED

====================== 1 passed, 47 deselected in 1.54s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (rope): Implement RoPE (2 points)

**Deliverable**: Implement a class `RotaryPositionalEmbedding` that applies RoPE to the input tensor.

The following interface is recommended:

`def __init__(self, theta: float, d_k: int, max_seq_len: int, device=None)` Construct the
RoPE module and create buffers if needed.

``` python
theta: float Θ value for the RoPE
d_k: int dimension of query and key vectors
max_seq_len: int Maximum sequence length that will be inputted
device: torch.device | None = None Device to store the buffer on
```

`def forward(self, x: torch.Tensor, token_positions: torch.Tensor) -> torch.Tensor` Process an input tensor of shape `(..., seq_len, d_k)` and return a tensor of the same shape. Note that you should tolerate x with an arbitrary number of batch dimensions. You should assume that the token positions are a tensor of shape `(..., seq_len)` specifying the token positions of `x` along the sequence dimension.

You should use the token positions to slice your (possibly precomputed) cos and sin tensors along the sequence dimension.

To test your implementation, complete [adapters.run_rope] and make sure it passes `uv run pytest -k test_rope`.

In [5]:
uv run pytest -k test_rope

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_model.py::test_rope PASSED

====================== 1 passed, 47 deselected in 0.14s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (softmax): Implement softmax (1 point)

**Deliverable**: Write a function to apply the `softmax` operation on a tensor. Your function should take two parameters: a tensor and a dimension $i$, and apply softmax to the $i$-th dimension of the input tensor. The output tensor should have the same shape as the input tensor, but its $i$-th dimension will now have a normalized probability distribution. Use the trick of subtracting the maximum value in the $i$-th dimension from all elements of the $i$-th dimension to avoid numerical stability issues.

To test your implementation, complete `[adapters.run_softmax]` and make sure it passes `uv run pytest -k test_softmax_matches_pytorch`.


In [6]:
uv run pytest -k test_softmax_matches_pytorch

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_nn_utils.py::test_softmax_matches_pytorch PASSED

====================== 1 passed, 47 deselected in 0.11s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (scaled_dot_product_attention): Implement scaled dot-product attention (5 points)

**Deliverable**: Implement the scaled dot-product attention function. Your implementation should handle keys and queries of shape `(batch_size, ..., seq_len, d_k)` and values of shape `(batch_size, ..., seq_len, d_v)`, where `...` represents any number of other batch-like dimensions (if provided). The implementation should return an output with the shape `(batch_size, ..., d_v)`. See section $3.3$ for a discussion on batch-like dimensions.

Your implementation should also support an optional user-provided boolean mask of shape `(seq_len, seq_len)`. The attention probabilities of positions with a mask value of `True` should collectively sum
to $1$, and the attention probabilities of positions with a mask value of `False` should be zero.
To test your implementation against our provided tests, you will need to implement the test adapter
at `[adapters.run_scaled_dot_product_attention]`.

`uv run pytest -k test_scaled_dot_product_attention` tests your implementation on third-order input tensors, while `uv run pytest -k test_4d_scaled_dot_product_attention` tests your implementation on fourth-order input tensors.

In [10]:
uv run pytest -k test_scaled_dot_product_attention

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_model.py::test_scaled_dot_product_attention PASSED

====================== 1 passed, 47 deselected in 1.49s =======================
Note: you may need to restart the kernel to use updated packages.


In [9]:
uv run pytest -k test_4d_scaled_dot_product_attention

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_model.py::test_4d_scaled_dot_product_attention PASSED

====================== 1 passed, 47 deselected in 1.58s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (multihead_self_attention): Implement causal multi-head self-attention (5 points)

**Deliverable**: Implement causal multi-head self-attention as a `torch.nn.Module`. Your implementation should accept (at least) the following parameters:

``` python
d_model: int Dimensionality of the Transformer block inputs.
num_heads: int Number of heads to use in multi-head self-attention.
```

Folllowing $\text{Vaswani et al. [2017]}$, set $d_k=d_v=\frac{d_{model}}h$. To test your implementation against our provided tests, implement the test adapter at `[adapters.run_multihead_self_attention]`. Then, run `uv run pytest -k test_multihead_self_attention` to test your implementation.

In [1]:
uv run pytest -k test_multihead_self_attention

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 46 deselected / 2 selected

tests/test_model.py::test_multihead_self_attention PASSED
tests/test_model.py::test_multihead_self_attention_with_rope PASSED

====================== 2 passed, 46 deselected in 1.51s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (transformer_block): Implement the Transformer block (3 points)

Implement the pre-norm Transformer block as described in $§3.5$ and illustrated in $\text{Figure} 2$. Your
Transformer block should accept (at least) the following parameters.

``` python
d_model: int Dimensionality of the Transformer block inputs.
num_heads: int Number of heads to use in multi-head self-attention.
d_ff: int Dimensionality of the position-wise feed-forward inner layer.
```

To test your implementation, implement the adapter `[adapters.run_transformer_block]`. Then run `uv run pytest -k test_transformer_block` to test your implementation.

**Deliverable**: Transformer block code that passes the provided tests.

In [2]:
uv run pytest -k test_transformer_block

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_model.py::test_transformer_block PASSED

====================== 1 passed, 47 deselected in 1.53s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (transformer_lm): Implementing the Transformer LM (3 points)

Time to put it all together! Implement the Transformer language model as described in $§3.1$ and illustrated in $\text{Figure} 1$. At minimum, your implementation should accept all the aforementioned construction parameters for the Transformer block, as well as these additional parameters:

``` python
vocab_size: int The size of the vocabulary, necessary for determining the dimensionality of the token embedding matrix.
context_length: int The maximum context length, necessary for determining the dimensionality of the position embedding matrix.
num_layers: int The number of Transformer blocks to use.
```

To test your implementation against our provided tests, you will first need to implement the test adapter at `[adapters.run_transformer_lm]`. Then, run `uv run pytest -k test_transformer_lm` to test your implementation.

**Deliverable**: A Transformer LM module that passes the above tests.

In [3]:
uv run pytest -k test_transformer_lm

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 46 deselected / 2 selected

tests/test_model.py::test_transformer_lm PASSED
tests/test_model.py::test_transformer_lm_truncated_input PASSED

====================== 2 passed, 46 deselected in 1.56s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (transformer_accounting): Transformer LM resource accounting (5 points)

Consider GPT-2 XL, which has the following configuration:

``` python
vocab_size : 50,257
context_length : 1,024
num_layers : 48
d_model : 1,600
num_heads : 25
d_ff : 6,400
```

Suppose we constructed our model using this configuration. How many trainable parameters
would our model have? Assuming each parameter is represented using single-precision floating
point, how much memory is required to just load this model?

**Deliverable**: A one-to-two sentence response.

---

``` markdown
**token_embeddings**: vocab_size * d_model = 50,257 * 1,600 = 80,411,200
**note**: embedding is is a **lookup**, but it is mathematically equivalent to a **matrix multiplication** with a **one-hot encoded vector**.

**layers**:
- **attn**:
  - q_proj: d_model × d_model = 1,600 × 1,600 = 2,560,000
  - k_proj: d_model × d_model = 1,600 × 1,600 = 2,560,000
  - v_proj: d_model × d_model = 1,600 × 1,600 = 2,560,000
  - output_proj: d_model × d_model = 1,600 × 1,600 = 2,560,000
- **ffn**:
  - w1: d_model × d_ff = 1,600 × 6,400 = 10,240,000
  - w2: d_ff × d_model = 6,400 × 1,600 = 10,240,000
  - w3: d_model × d_ff = 1,600 × 6,400 = 10,240,000
  **ln1**: d_model = 1600
  **ln2**: d_model = 1600
- 40,963,200 x num_layers = 40,963,200 x 48 = 1,966,233,600

**ln_final**：d_model = 1,600

**lm_head**: vocab_size * d_model = 50,257 * 1,600 = 80,411,200
```

<span style="background-color: #29B6F6; color: black">
2,127,058,600 parameters, × 4 bytes = 7.92 GB
</span>

---

Identify the matrix multiplies required to complete a forward pass of our GPT-2 XL-shaped
model. How many `FLOPs` do these matrix multiplies require in total? Assume that our input
sequence has context_length tokens.

**Deliverable**: A list of matrix multiplies (with descriptions), and the total number of `FLOPs`
required.

---

``` markdown
**Transformer Block**
- MHA
- - q_proj: (1024, 1600) × (1600, 1600) → FLOPs: 2 × 1024 × 1600 × 1600 = 5,242,880,000
- - k_proj: (1024, 1600) × (1600, 1600) → FLOPs: 2 × 1024 × 1600 × 1600 = 5,242,880,000
- - v_proj: (1024, 1600) × (1600, 1600) → FLOPs: 2 × 1024 × 1600 × 1600 = 5,242,880,000
- - attention scores: (25, 1024, 64) × (25, 64, 1024) → FLOPs: 2 × 25 × 1024 × 64 × 1024 = 3,355,443,200
- - attention: (25, 1024, 1024) × (25, 1024, 64) → FLOPs: 2 × 25 × 1024 × 1024 × 64 = 3,355,443,200
- - output_proj: (1024, 1600) × (1600, 1600) → FLOPs: 2 × 1024 × 1600 × 1600 = 5,242,880,000
- FFN
- - w1: (1024, 1600) × (1600, 6400) → FLOPs: 2 × 1024 × 1600 × 6400 = 20,971,520,000
- - w3: (1024, 1600) × (1600, 6400) → FLOPs: 2 × 1024 × 1600 × 6400 = 20,971,520,000
- - w2: (1024, 6400) × (6400, 1600) → FLOPs: 2 × 1024 × 6400 × 1600 = 20,971,520,000
- 90,596,966,400 * 48 = 4,348,654,387,200

**LM Head**: (1024, 1600) × (1600, 50257) → FLOPs: 2 × 1024 × 1600 × 50257 = 164,681,932,800
```

<span style="background-color: #29B6F6; color: black">
4,513,297,920,000 FLOPs
</span>

---

Based on your analysis above, which parts of the model require the most `FLOPs`?

**Deliverable**: A one-to-two sentence response.

---

<span style="background-color: #29B6F6; color: black">
FFN, as it projects the hidden state from d_model to a larger intermediate dimension d_ff (4x in this case), and then projects it back, which involves matrix multiplication operations much larger than the self attention part.
</span>

---

Repeat your analysis with GPT-2 small (12 layers, 768 d_model, 12 heads), GPT-2 medium (24
layers, 1024 d_model, 16 heads), and GPT-2 large (36 layers, 1280 d_model, 20 heads). As the
model size increases, which parts of the Transformer LM take up proportionally more or less of
the total `FLOPs`?

**Deliverable**: For each model, provide a breakdown of model components and its associated
`FLOPs` (as a proportion of the total `FLOPs` required for a forward pass). In addition, provide a
one-to-two sentence description of how varying the model size changes the proportional `FLOPs`
of each component.

---

| model            | d_model (d) | FFN FLOPs         | Attention FLOPs | FFN %  | Attention % |
| ---------------- | ----------- | ----------------- | --------------- | ------ | ----------- |
| **GPT-2 small**  | 768         | 11,509,596,160    | 4,731,174,912   | ~64.3% | ~35.7%      |
| **GPT-2 medium** | 1024        | 25,769,803,776    | 12,884,901,888  | ~66.7% | ~33.3%      |
| **GPT-2 large**  | 1280        | 40,265,318,400    | 18,790,481,920  | ~68.2% | ~31.8%      |
| **GPT-2 XL**     | 1600        | 62,914,560,000    | 27,682,406,400  | ~69.4% | ~30.6%      |


<span style="background-color: #29B6F6; color: black">
The proportion of FFN in total FLOPs will relatively increase, while the proportion of self attention will relatively decrease
</span>

---

Take GPT-2 XL and increase the context length to 16,384. How does the total `FLOPs` for one
forward pass change? How do the relative contribution of `FLOPs` of the model components
change?

**Deliverable**: A one-to-two sentence response.

---

``` markdown
**Attention** increase sequence length dimension by 16 times, FLOPs increase by 16 times
**FFN**: Increase sequence length dimension by 16 times, FLOPs increase by 16 times
**Total**L FLOPs increase by approximately 16 times

**FFN** still dominates, but the proportion of sequence length related calculations (Q × K and attention × V) for attention will slightly increase, as the complexity of these operations is O(n^2), while other operations are O(n).
```

---

## 4. Training a Transformer LM

Maybe useful: https://zhuanlan.zhihu.com/p/1927132746578366957

### Problem (cross_entropy): Implement Cross entropy

**Deliverable**: Write a function to compute the cross entropy loss, which takes in predicted logits $(o_i)$ and targets $(x_{i+1})$ and computes the cross entropy $ℓi = − \text{log} (\text{softmax}(o_i)[x_{i+1}])$. Your function should handle the following:

- Subtract the largest element for numerical stability.
- Cancel out log and exp whenever possible.
- Handle any additional batch dimensions and return the $\text{average}$ across the batch. As with section $3.3$, we assume batch-like dimensions always come first, before the vocabulary size dimension.

Implement `[adapters.run_cross_entropy]`, then run `uv run pytest -k test_cross_entropy` to test your implementation.


In [1]:
uv run pytest -k test_cross_entropy

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_nn_utils.py::test_cross_entropy PASSED

====================== 1 passed, 47 deselected in 0.18s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (learning_rate_tuning): Tuning the learning rate (1 point)

As we will see, one of the hyperparameters that affects training the most is the learning rate. Let’s see that in practice in our toy example. Run the `SGD` example above with three other values for the learning rate: `1e1`, `1e2`, and `1e3`, for just 10 training iterations. What happens with the loss for each of these learning rates? Does it decay faster, slower, or does it diverge (i.e., increase over the course of training)?

**Deliverable**: A one-two sentence response with the behaviors you observed.

In [ ]:
from collections.abc import Callable, Iterable
from typing import Optional
import torch
import math

class SGD(torch.optim.Optimizer):
    def __init__(self, params, lr=1e-3):
        if lr < 0:
            raise ValueError(f"Invalid learning rate: {lr}")
        defaults = {"lr": lr}
        super().__init__(params, defaults)
        
    def step(self, closure: Optional[Callable] = None):
        loss = None if closure is None else closure()
        for group in self.param_groups:
            lr = group["lr"] # Get the learning rate.
            for p in group["params"]:
                if p.grad is None:
                    continue
                
            state = self.state[p] # Get state associated with p.
            t = state.get("t", 0) # Get iteration number from the state, or initial value.
            grad = p.grad.data # Get the gradient of loss with respect to p.
            p.data -= lr / math.sqrt(t + 1) * grad # Update weight tensor in-place.
            state["t"] = t + 1 # Increment iteration number.
        
        return loss

In [3]:
def train(lr):
    weights = torch.nn.Parameter(5 * torch.randn((10, 10)))
    opt = SGD([weights], lr=lr)

    for t in range(10):
        opt.zero_grad() # Reset the gradients for all learnable parameters.
        loss = (weights**2).mean() # Compute a scalar loss value.
        print(loss.cpu().item())
        loss.backward() # Run backward pass, which computes gradients.
        opt.step() # Run optimizer step.

In [4]:
train(1e1)

22.122913360595703
14.15866470336914
10.437161445617676
8.165966033935547
6.614432334899902
5.484122276306152
4.625129222869873
3.952305316925049
3.413126230239868
2.973212242126465


In [5]:
train(1e2)

29.372173309326172
29.372173309326172
5.039466857910156
0.12060579657554626
2.271793980606257e-16
2.5320552761258588e-18
8.526320552724894e-20
5.0791882485921435e-21
4.357255665721727e-22
4.841394938711859e-23


In [6]:
train(1e3)

32.06910705566406
11576.94921875
1999521.125
222425376.0
18016452608.0
1137044619264.0
58372153933824.0
2511419849310208.0
9.25655732500562e+16
2.9723837255238287e+18


<span style="background-color: #29B6F6; color: black">
a learning rate of 1e1, the loss decays slowly, while a learning rate of 1e2 causes the loss to decay very rapidly. A learning rate of 1e3 is too high and causes the loss to diverge, increasing exponentially throughout training.
</span>

### Problem (adamw): Implement AdamW (2 points)

![](./data/AdamW.png)

**Deliverable**: Implement the AdamW optimizer as a subclass of `torch.optim.Optimizer`. Your class should take the learning rate $\alpha$ in `__init__`, as well as the $\beta$, $\epsilon$ and $\lambda$ hyperparameters. To help you keep state, the base Optimizer class gives you a dictionary `self.state`, which maps `nn.Parameter` objects to a dictionary that stores any information you need for that parameter (for AdamW, this would be the moment estimates). Implement `[adapters.get_adamw_cls]` and make sure it passes `uv run pytest -k test_adamw`.

In [3]:
uv run pytest -k test_adamw

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_optimizer.py::test_adamw PASSED

====================== 1 passed, 47 deselected in 2.30s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (adamwAccounting): Resource accounting for training with AdamW (2 points)

Let us compute how much memory and compute running AdamW requires. Assume we are using `float32` for every tensor.

How much peak memory does running `AdamW` require? Decompose your answer based on the memory usage of the parameters, activations, gradients, and optimizer state. Express your answer in terms of the batch_size and the model hyperparameters (`vocab_size`, `context_length`, `num_layers`, `d_model`, `num_heads`). Assume `d_ff = 4 × d_model`.

For simplicity, when calculating memory usage of activations, consider only the following components:
- Transformer block
  - RMSNorm(s)
  - Multi-head self-attention sublayer: $QKV$ projections, $Q^TK$ matrix multiply, softmax, weighted sum of values, output projection.
  - Position-wise feed-forward: $W_1$ matrix multiply, $\text{SiLU}$, $W_2$ matrix multiply
- final RMSNorm
- output embedding
- cross-entropy on logits

**Deliverable**: An algebraic expression for each of parameters, activations, gradients, and optimizer state, as well as the total.

---
* **Embedding:**

  * `token_embeddings`: `vocab_size × d_model`
  * `lm_head`: `d_model × vocab_size`

* **Per Transformer Block:**

  * Attention: `4 × d_model²`
  * Feedforward (SwiGLU): `3 × d_model × d_ff`
  * LayerNorms: `2 × d_model`
  * Total per block: `4d_model² + 3d_model × d_ff + 2d_model`

* **Final LayerNorm:** `d_model`

* **Total Parameters:**
  Given `d_ff = 4 × d_model`,

  ```
  P = 2 × vocab_size × d_model  
      + num_layers × (16 × d_model² + 2 × d_model)  
      + d_model
  ```

* **Parameter Memory (float32):**
  `4 × P` bytes

**Activation Memory (Peak)**

* **Within a Transformer Block:**

  * Inputs/outputs: `batch_size × context_length × d_model`
  * **Attention:**

    * `Q, K, V`: `3 × batch_size × context_length × d_model`
    * `QKᵀ` (attention scores): `batch_size × num_heads × context_length²` <- *dominant*
  * **Feedforward (SwiGLU):**

    * Peak: `2 × batch_size × context_length × d_ff = 8 × batch_size × context_length × d_model`

* **Final Output:**

  * `lm_head` logits: `batch_size × context_length × vocab_size`
  * Cross-entropy reuses logits

* **Peak Activation Memory:**
  `4 × (batch_size × context_length × vocab_size)` bytes

**Gradient Memory**

* Same shape as parameters
  → `4 × P` bytes

**Optimizer State (AdamW)**

* Two float32 tensors per parameter (`exp_avg`, `exp_avg_sq`)
  → `8 × P` bytes

**Total Peak Memory**

```
Total = 4 × P                          // parameters
      + 4 × batch_size × context_length × vocab_size   // activations
      + 4 × P                          // gradients
      + 8 × P                          // optimizer states
      = 16 × P + 4 × batch_size × context_length × vocab_size bytes
```

Where:

```
P = 2 × vocab_size × d_model  
  + num_layers × (16 × d_model² + 2 × d_model)  
  + d_model
```

---

Instantiate your answer for a GPT-2 XL-shaped model to get an expression that only depends on the `batch_size`. What is the maximum batch size you can use and still fit within 80GB memory?

**Deliverable**: An expression that looks like $a \cdot$ `batch_size` $+ b$ for numerical values $a$, $b$, and a number representing the maximum batch size.

---

``` python
vocab_size = 50,257  
d_model = 1,600  
num_layers = 48  
d_ff = 4 × d_model = 6,400  
context_length = 1,024
```

$$
\begin{aligned}
P &= 2 \cdot 50257 \cdot 1600 + 48 \cdot (16 \cdot 1600^2 + 2 \cdot 1600) + 1600 \\
  &= \boxed{2,127,057,600} \text{ parameters}
\end{aligned}
$$

$$
\begin{aligned}
a &= 4 \cdot T \cdot V = 4 \cdot 1024 \cdot 50257 = \boxed{205,852,672} \\
b &= 16 \cdot P = 16 \cdot 2,127,057,600 = \boxed{34,032,921,600}
\end{aligned}
$$

Therefore, we have:

$$
a \cdot B + b \leq 85,899,345,920
$$

$$
\boxed{\text{PeakMemory} = 205{,}852{,}672 \cdot \text{batch\_size} + 34{,}032{,}921{,}600}
$$

max batch size:

$$
\boxed{251}
$$

---

How many `FLOPs` does running one step of `AdamW` take?

**Deliverable**: An algebraic expression, with a brief justification.

---
Each parameter update involves:
  1. **Weight decay:** 2 FLOPs (multiply, subtract)
  2. **First moment update:** 4 FLOPs (2 multiplies, 1 subtract, 1 add)
  3. **Second moment update:** 5 FLOPs (square, 2 multiplies, subtract, add)
  4. **Final update:** 5 FLOPs (sqrt, add, divide, multiply, subtract)

* **Total:** 16 FLOPs per parameter (ignoring per-step `lr_t` computation)

* For GPT-2 XL (≈2.13B params):

  ```
  FLOPs = 16 × 2,127,057,600 = 34.03 billion
  ```

---

Model `FLOPs` utilization (MFU) is defined as the ratio of observed throughput (tokens per second) relative to the hardware’s theoretical peak FLOP throughput $\text{[Chowdhery et al., 2022]}$. An NVIDIA A100 GPU has a theoretical peak of `19.5 teraFLOP/s` for `float32` operations. Assuming
you are able to get 50% MFU, how long would it take to train a GPT-2 XL for 400K steps and a batch size of 1024 on a single A100? Following $\text{Kaplan et al. [2020]}$ and $\text{Hoffmann et al. [2022]}$, assume that the backward pass has twice the `FLOPs` of the forward pass.

**Deliverable**: The number of days training would take, with a brief justification.

---

* **FLOPs per step:**

  * Forward: `4.51 × 10¹²`
  * Backward: `2 × forward = 9.03 × 10¹²`
  * Total per step: `13.54 × 10¹²`

* **Total steps:** `400,000`

* **Total training FLOPs:**

  ```
  400,000 × 13.54 × 10¹² = 5.42 × 10¹⁸
  ```

* **GPU (A100) throughput:**

  * Peak: `19.5 × 10¹²` FLOP/s
  * Utilization (MFU): 50% → Effective: `9.75 × 10¹²` FLOP/s

* **Training time:**

  ```
  Seconds = 5.42 × 10¹⁸ ÷ 9.75 × 10¹² ≈ 555,483
  Days = 555,483 ÷ 86,400 ≈ 6.43
  ```

---

### Problem (learning_rate_schedule): Implement cosine learning rate schedule with warmup

Write a function that takes $t, \alpha_\text{max}, \alpha_\text{min}, T_w$ and $T_c$, and returns the learning rate $\alpha_t$ according to the scheduler defined above. Then implement `[adapters.get_lr_cosine_schedule]` and make sure it passes `uv run pytest -k test_get_lr_cosine_schedule`.

In [1]:
uv run pytest -k test_get_lr_cosine_schedule

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_optimizer.py::test_get_lr_cosine_schedule PASSED

====================== 1 passed, 47 deselected in 2.10s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (gradient_clipping): Implement gradient clipping (1 point)

Write a function that implements gradient clipping. Your function should take a list of parameters and a maximum $\ell_2$-norm. It should modify each parameter gradient in place. Use $\epsilon$ = `10−6` (the PyTorch default). Then, implement the adapter `[adapters.run_gradient_clipping]` and make sure it passes `uv run pytest -k test_gradient_clipping`.

In [2]:
uv run pytest -k test_gradient_clipping

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_nn_utils.py::test_gradient_clipping PASSED

====================== 1 passed, 47 deselected in 0.12s =======================
Note: you may need to restart the kernel to use updated packages.


## 5. Training loop

### Problem (data_loading): Implement data loading (2 points)

**Deliverable**: Write a function that takes a numpy array x (integer array with token IDs), a `batch_size`, a `context_length` and a PyTorch device string (e.g., 'cpu' or 'cuda:0'), and returns a pair of tensors: the sampled input sequences and the corresponding next-token targets. Both tensors should have shape (batch_size, context_length) containing token IDs, and both should be placed on the requested device. To test your implementation against our provided tests, you will first need to implement the test adapter at `[adapters.run_get_batch]`. Then, run `uv run pytest -k test_get_batch` to test your implementation.

In [1]:
uv run pytest -k test_get_batch

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_data.py::test_get_batch PASSED

====================== 1 passed, 47 deselected in 0.46s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (checkpointing): Implement model checkpointing (1 point)

Implement the following two functions to load and save checkpoints:

`def save_checkpoint(model, optimizer, iteration, out)` should dump all the state from the
first three parameters into the file-like object out. You can use the state_dict method of both the model and the optimizer to get their relevant states and use `torch.save(obj, out)` to dump obj into out (PyTorch supports either a path or a file-like object here). A typical choice is to have obj be a dictionary, but you can use whatever format you want as long as you can load your checkpoint later.

This function expects the following parameters:

``` python
model: torch.nn.Module
optimizer: torch.optim.Optimizer
iteration: int
out: str | os.PathLike | typing.BinaryIO | typing.IO[bytes]
```

`def load_checkpoint(src, model, optimizer)` should load a checkpoint from src (path or filelike object), and then recover the model and optimizer states from that checkpoint. Your function should return the iteration number that was saved to the checkpoint. You can use `torch.load(src)` to recover what you saved in your `save_checkpoint` implementation, and the `load_state_dict` method in both the model and optimizers to return them to their previous states.

This function expects the following parameters:

``` python
src: str | os.PathLike | typing.BinaryIO | typing.IO[bytes]
model: torch.nn.Module
optimizer: torch.optim.Optimizer
```

Implement the `[adapters.run_save_checkpoint]` and `[adapters.run_load_checkpoint]` adapters, and make sure they pass `uv run pytest -k test_checkpointing`.


In [1]:
uv run pytest -k test_checkpointing

============================= test session starts =============================
platform win32 -- Python 3.11.0rc2, pytest-8.3.5, pluggy-1.5.0
rootdir: e:\Code\CS336\assignment1-basics
configfile: pyproject.toml
plugins: jaxtyping-0.3.1
collected 48 items / 47 deselected / 1 selected

tests/test_serialization.py::test_checkpointing PASSED

====================== 1 passed, 47 deselected in 1.62s =======================
Note: you may need to restart the kernel to use updated packages.


### Problem (training_together): Put it together (4 points)

**Deliverable**: Write a script that runs a training loop to train your model on user-provided input.

In particular, we recommend that your training script allow for (at least) the following:

- Ability to configure and control the various model and optimizer hyperparameters.
- Memory-efficient loading of training and validation large datasets with np.memmap.
- Serializing checkpoints to a user-provided path.
- Periodically logging training and validation performance (e.g., to console and/or an external service like Weights and Biases).

---

## 6. Generating text

Maybe useful: https://zhuanlan.zhihu.com/p/1927311888515073257

### Problem (decoding): Decoding (3 points)

**Deliverable**: Implement a function to decode from your language model. We recommend that you support the following features:

- Generate completions for a user-provided prompt (i.e., take in some x1...t and sample a completion until you hit an `<|endoftext|>` token).
- Allow the user to control the maximum number of generated tokens.
- Given a desired temperature value, apply softmax temperature scaling to the predicted next-word distributions before sampling.
- Top-p sampling ($\text{Holtzman et al., 2020}$; also referred to as nucleus sampling), given a user-specified threshold value.

## 7. Experiments

---